# Fouille données

In [6]:
!pip install pandas numpy matplotlib seaborn folium scikit-learn

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from collections import Counter

%matplotlib inline
sns.set(style="whitegrid")

# Importation des données

In [8]:
df = pd.read_csv('flickr_data2.csv', on_bad_lines='skip', sep=",")
print("Nombre de photos au départ :  " + str(len(df)))

Nombre de photos au départ :  420240


C:\Users\jerom\AppData\Local\Temp\ipykernel_22724\2839054439.py:1: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('flickr_data2.csv', on_bad_lines='skip', sep=",")


In [15]:
# Affichage du tableau des données brutes avant nettoyage
display(df)

,id,user,lat,long,tags,title,date_taken_minute,date_taken_hour,date_taken_day,date_taken_month,date_taken_year,date_upload_minute,date_upload_hour,date_upload_day,date_upload_month,date_upload_year,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,4395181099,30624617@N03,45.754858,4.821710,"chair,lyon,rhône,chaise,rhônealpes",Chaises avec vue,11.0,15,28,2,2010,23,20,28.0,2,2010.0,NaN,NaN,NaN
1,4394748717,35853470@N00,45.753270,4.862953,NaN,NaN,51.0,17,28,2,2010,52,17,28.0,2,2010.0,NaN,NaN,NaN
2,4394694699,11817998@N05,45.760655,4.846564,"365,iphone",59/365 - R46 V103 B163,29.0,17,28,2,2010,33,17,28.0,2,2010.0,NaN,NaN,NaN
3,4394803790,11545749@N06,45.784000,4.874072,"nin,nineinchnails,gift,screening,toiou,avott",2010-01-29 Toiou Avott Lyon,15.0,20,28,1,2010,38,12,28.0,2,2010.0,NaN,NaN,NaN
4,4394803554,11545749@N06,45.784000,4.874072,"lyon,nin,nineinchnails,gift,screening,toiou,avott",2010-01-28 Toiou Avott Lyon,10.0,20,28,1,2010,38,12,28.0,2,2010.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420235,44402328654,90493526@N00,45.758316,4.825197,"europe,france,lyon,croixrousse,streetart,wheat...","Pasted paper by Big Ben [Lyon, France]",18.0,17,30,9,2018,11.0,23,5.0,10,2018.0,NaN,NaN,NaN
420236,44210748275,144146684@N04,45.762635,4.837299,NaN,white blood,36.0,16,5,10,2018,41.0,22,5.0,10,2018.0,NaN,NaN,NaN
420237,45122361361,95450872@N03,45.763657,4.836012,"auvergnerhônealpes,rhône,lyonnais,valléedurhôn...",Lyon - Porte Passage de l'Argue,48.0,19,27,9,2018,40.0,22,5.0,10,2018.0,NaN,NaN,NaN
420238,45073351222,95450872@N03,45.763657,4.836012,"auvergnerhônealpes,rhône,lyonnais,valléedurhôn...",Lyon - Passage de l'Argue,48.0,19,27,9,2018,29.0,22,5.0,10,2018.0,NaN,NaN,NaN


In [18]:
# Vérification et affichage des données incohérentes dans df original (avant nettoyage)
print("Filtrage des données avec dates incohérentes :")
condition_invalide = (
    (df['date_taken_hour'] < 0) | (df['date_taken_hour'] > 23) |
    (df['date_taken_month'] < 1) | (df['date_taken_month'] > 12) |
    (df['date_taken_day'] < 1) | (df['date_taken_day'] > 31)
)
df_incoherentes = df[condition_invalide]
print(f"Nombre total de données incohérentes : {len(df_incoherentes)}")
print("Affichage du tableau des données incohérentes :")
display(df_incoherentes)

Filtrage des données avec dates incohérentes :
Nombre total de données incohérentes : 116
Affichage du tableau des données incohérentes :


,id,user,lat,long,tags,title,date_taken_minute,date_taken_hour,date_taken_day,date_taken_month,date_taken_year,date_upload_minute,date_upload_hour,date_upload_day,date_upload_month,date_upload_year,Unnamed: 16,Unnamed: 17,Unnamed: 18
42366,5464485473,35635047@N03,45.765517,4.766510,"lundimatin,lyondefi38nuit",une lundi matin comme tout les autre ;-(25,6.0,21,2,2011,11,15,21,2.0,2011,NaN,NaN,NaN,NaN
85950,6674970791,29713277@N02,45.753948,4.788145,"portrait,throughtheleaves,autraversdesfeuillages",NaN,2012.0,29,12,10,9,NaN,47,21.0,10,1.0,2011.0,NaN,NaN
90872,6674970791,29713277@N02,45.753948,4.788145,"portrait,throughtheleaves,autraversdesfeuillages",NaN,2012.0,29,12,10,9,NaN,47,21.0,10,1.0,2011.0,NaN,NaN
98816,7387000024,38586649@N00,45.779196,4.853596,"uploaded:by=flicksquare,foursquare:venue=4b851...",Beautiful weather in #Lyon,2012.0,46,11,17,6,),46,17.0,17,6.0,2012.0,NaN,NaN
102366,7583546064,37290448@N04,45.772827,4.841065,"city,blackandwhite,france,water,river,french,c...",Bridge,2012.0,38,11,7,6,Rhône,5,18.0,16,7.0,2012.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390775,33105832478,53987060@N02,45.771063,4.829993,NaN,Looking from the Roman arenas to the Basilica ...,2019.0,57,11,16,11,-),45,13.0,4,2.0,2018.0,NaN,NaN
391474,31807499637,53987060@N02,45.762097,4.821281,NaN,Inside the Basilica of Notre-Dame de Fourvière...,2019.0,55,12,15,11,-),25,8.0,15,1.0,2018.0,NaN,NaN
391596,31807499637,53987060@N02,45.762097,4.821281,NaN,Inside the Basilica of Notre-Dame de Fourvière...,2019.0,55,12,15,11,-),25,8.0,15,1.0,2018.0,NaN,NaN
395508,47103124882,15917364@N02,45.776985,4.833829,"lyon,rhône,croixrousse,escaliers,rue,personne,...",Lyon - En solitaire à la Croix-Rousse,2019.0,54,17,18,2,NaN,29,12.0,20,2.0,2019.0,NaN,NaN


In [19]:
# Analyse profonde des données incohérentes
print("Analyse des données incohérentes :")
print(f"Colonnes disponibles : {list(df_incoherentes.columns)}")

# Statistiques descriptives des colonnes de date pour les incohérentes
print("\nStatistiques des colonnes de date dans les données incohérentes :")
date_cols = ['date_taken_year', 'date_taken_month', 'date_taken_day', 'date_taken_hour', 'date_taken_minute']
print(df_incoherentes[date_cols].describe())

# Vérifier les types de données
print("\nTypes de données dans df_incoherentes :")
print(df_incoherentes[date_cols].dtypes)

# Examiner quelques exemples spécifiques
print("\nExemples de lignes incohérentes (premières 10) :")
display(df_incoherentes[date_cols].head(10))

# Chercher des patterns : par exemple, heures >23, voir si elles pourraient être des minutes
heures_sup_23 = df_incoherentes[df_incoherentes['date_taken_hour'] > 23]
print(f"\nLignes avec heure >23 : {len(heures_sup_23)}")
if len(heures_sup_23) > 0:
    print("Valeurs d'heure >23 :")
    print(heures_sup_23['date_taken_hour'].unique())
    print("Exemples :")
    display(heures_sup_23[['date_taken_year', 'date_taken_month', 'date_taken_day', 'date_taken_hour', 'date_taken_minute']].head(5))

# Mois >12
mois_sup_12 = df_incoherentes[df_incoherentes['date_taken_month'] > 12]
print(f"\nLignes avec mois >12 : {len(mois_sup_12)}")
if len(mois_sup_12) > 0:
    print("Valeurs de mois >12 :")
    print(mois_sup_12['date_taken_month'].unique())
    print("Exemples :")
    display(mois_sup_12[['date_taken_year', 'date_taken_month', 'date_taken_day', 'date_taken_hour', 'date_taken_minute']].head(5))

# Jours >31
jours_sup_31 = df_incoherentes[df_incoherentes['date_taken_day'] > 31]
print(f"\nLignes avec jour >31 : {len(jours_sup_31)}")
if len(jours_sup_31) > 0:
    print("Valeurs de jour >31 :")
    print(jours_sup_31['date_taken_day'].unique())
    print("Exemples :")
    display(jours_sup_31[['date_taken_year', 'date_taken_month', 'date_taken_day', 'date_taken_hour', 'date_taken_minute']].head(5))

# Vérifier si les valeurs semblent décalées (par exemple, heure dans minute, etc.)
print("\nVérification de décalage possible :")
# Par exemple, si heure >23, voir si minute est valide
if len(heures_sup_23) > 0:
    print("Pour les heures >23, vérifier les minutes :")
    print(heures_sup_23['date_taken_minute'].describe())

# De même pour mois
if len(mois_sup_12) > 0:
    print("Pour les mois >12, vérifier les jours :")
    print(mois_sup_12['date_taken_day'].describe())

# Chercher des NaN ou valeurs manquantes
print(f"\nNombre de NaN par colonne dans les incohérentes :")
print(df_incoherentes[date_cols].isnull().sum())

# Voir si les valeurs sont des floats ou strings étranges
print("\nExemples de valeurs uniques pour diagnostiquer :")
for col in date_cols:
    unique_vals = df_incoherentes[col].dropna().unique()
    print(f"{col} : {unique_vals[:10]}... (total unique: {len(unique_vals)})")  # Premiers 10 pour aperçu

Analyse des données incohérentes :
Colonnes disponibles : ['id', 'user', 'lat', 'long', 'tags', 'title', 'date_taken_minute', 'date_taken_hour', 'date_taken_day', 'date_taken_month', 'date_taken_year', 'date_upload_minute', 'date_upload_hour', 'date_upload_day', 'date_upload_month', 'date_upload_year', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18']

Statistiques des colonnes de date dans les données incohérentes :
       date_taken_year  date_taken_month  date_taken_day  date_taken_hour  \
count        116.00000        116.000000      116.000000       116.000000   
mean           7.62069         35.353448       46.982759        69.517241   
std            3.93569        185.253446      261.574496       258.962192   
min            1.00000          2.000000        2.000000         4.000000   
25%            6.00000         11.000000       10.000000        26.000000   
50%            8.00000         18.000000       12.000000        39.000000   
75%            9.00000         27.000000     

,date_taken_year,date_taken_month,date_taken_day,date_taken_hour,date_taken_minute
42366,11,2011,2,21,6.0
85950,9,10,12,29,2012.0
90872,9,10,12,29,2012.0
98816,6,17,11,46,2012.0
102366,6,7,11,38,2012.0
102720,7,8,16,49,2012.0
103681,6,17,11,46,2012.0
106983,6,7,11,38,2012.0
107337,7,8,16,49,2012.0
108300,6,17,11,46,2012.0



Lignes avec heure >23 : 92
Valeurs d'heure >23 :
[  29   46   38   49   44   28 2013   31   26   27   54   43   42   34
   48   33   24   47   39   32   57   55]
Exemples :


,date_taken_year,date_taken_month,date_taken_day,date_taken_hour,date_taken_minute
85950,9,10,12,29,2012.0
90872,9,10,12,29,2012.0
98816,6,17,11,46,2012.0
102366,6,7,11,38,2012.0
102720,7,8,16,49,2012.0



Lignes avec mois >12 : 84
Valeurs de mois >12 :
[2011   17   27   19   31   28   38   26   25   24   18   29   16   22
   15]
Exemples :


,date_taken_year,date_taken_month,date_taken_day,date_taken_hour,date_taken_minute
42366,11,2011,2,21,6.0
98816,6,17,11,46,2012.0
103681,6,17,11,46,2012.0
108300,6,17,11,46,2012.0
108687,9,27,20,44,2012.0



Lignes avec jour >31 : 2
Valeurs de jour >31 :
[2013]
Exemples :


,date_taken_year,date_taken_month,date_taken_day,date_taken_hour,date_taken_minute
137218,15,38,2013,5,16.0
142450,15,38,2013,5,16.0



Vérification de décalage possible :
Pour les heures >23, vérifier les minutes :
count      92.000000
mean     1971.695652
std       294.793343
min         5.000000
25%      2013.000000
50%      2016.000000
75%      2017.000000
max      2019.000000
Name: date_taken_minute, dtype: float64
Pour les mois >12, vérifier les jours :
count      84.000000
mean       60.011905
std       306.869561
min         2.000000
25%        10.000000
50%        11.000000
75%        17.000000
max      2013.000000
Name: date_taken_day, dtype: float64

Nombre de NaN par colonne dans les incohérentes :
date_taken_year      0
date_taken_month     0
date_taken_day       0
date_taken_hour      0
date_taken_minute    1
dtype: int64

Exemples de valeurs uniques pour diagnostiquer :
date_taken_year : [11  9  6  7  1  3 15 25 10  2]... (total unique: 14)
date_taken_month : [2011   10   17    7    8   27   19   31   28   38]... (total unique: 23)
date_taken_day : [   2   12   11   16   20    6   14 2013   28    3]... 

In [20]:
# Tentative de correction automatique du décalage de colonnes
def corriger_decalage(row):
    # Copie des valeurs actuelles
    y, m, d, h, mn = row['date_taken_year'], row['date_taken_month'], row['date_taken_day'], row['date_taken_hour'], row['date_taken_minute']

    # Si minute est une année (entre 2000 et 2030), décalage vers la droite détecté
    if pd.notna(mn) and 2000 <= mn <= 2030:
        # Décalage : year <- minute, minute <- hour, hour <- day, day <- month, month <- year
        new_y = int(mn)
        new_mn = h % 60  # minute de 0-59
        new_h = d % 24   # heure de 0-23
        new_d = m % 31   # jour approximatif 1-31
        new_m = y % 12   # mois 1-12
        if new_m == 0: new_m = 12
        if new_d == 0: new_d = 1
        return pd.Series([new_y, new_m, new_d, new_h, new_mn])

    # Si month >12 et semble être une année, décalage différent
    elif m > 12 and m < 2030:
        # Suppose month est year, alors décalage inverse
        new_y = int(m)
        new_m = y % 12
        if new_m == 0: new_m = 12
        new_d = d % 31
        if new_d == 0: new_d = 1
        new_h = h % 24
        new_mn = mn if pd.notna(mn) else 0
        new_mn = new_mn % 60
        return pd.Series([new_y, new_m, new_d, new_h, new_mn])

    # Si day >31 et semble année
    elif d > 31 and d < 2030:
        new_y = int(d)
        new_d = m % 31
        if new_d == 0: new_d = 1
        new_m = y % 12
        if new_m == 0: new_m = 12
        new_h = h % 24
        new_mn = mn if pd.notna(mn) else 0
        new_mn = new_mn % 60
        return pd.Series([new_y, new_m, new_d, new_h, new_mn])

    # Sinon, essayer de corriger seulement les invalides
    else:
        new_y = y
        new_m = m if 1 <= m <= 12 else (m % 12) if m % 12 != 0 else 12
        new_d = d if 1 <= d <= 31 else (d % 31) if d % 31 != 0 else 1
        new_h = h if 0 <= h <= 23 else (h % 24)
        new_mn = mn if pd.notna(mn) and 0 <= mn <= 59 else (mn % 60) if pd.notna(mn) else 0
        return pd.Series([new_y, new_m, new_d, new_h, new_mn])

# Appliquer la correction
df_incoherentes_corrected = df_incoherentes.copy()
df_incoherentes_corrected[['date_taken_year', 'date_taken_month', 'date_taken_day', 'date_taken_hour', 'date_taken_minute']] = df_incoherentes.apply(corriger_decalage, axis=1)

print("Après correction automatique :")
print("Exemples corrigés :")
display(df_incoherentes_corrected[['date_taken_year', 'date_taken_month', 'date_taken_day', 'date_taken_hour', 'date_taken_minute']].head(10))

# Vérifier combien sont maintenant valides
condition_valide_apres = (
    (df_incoherentes_corrected['date_taken_hour'] >= 0) & (df_incoherentes_corrected['date_taken_hour'] <= 23) &
    (df_incoherentes_corrected['date_taken_month'] >= 1) & (df_incoherentes_corrected['date_taken_month'] <= 12) &
    (df_incoherentes_corrected['date_taken_day'] >= 1) & (df_incoherentes_corrected['date_taken_day'] <= 31)
)
valides_apres = df_incoherentes_corrected[condition_valide_apres]
print(f"\nNombre de lignes maintenant valides après correction : {len(valides_apres)} / {len(df_incoherentes_corrected)}")

# Afficher les lignes corrigées valides
if len(valides_apres) > 0:
    print("Exemples de lignes corrigées et maintenant valides :")
    display(valides_apres[['id', 'date_taken_year', 'date_taken_month', 'date_taken_day', 'date_taken_hour', 'date_taken_minute']].head(10))

Après correction automatique :
Exemples corrigés :


,date_taken_year,date_taken_month,date_taken_day,date_taken_hour,date_taken_minute
42366,2011.0,11.0,2.0,21.0,6.0
85950,2012.0,9.0,10.0,12.0,29.0
90872,2012.0,9.0,10.0,12.0,29.0
98816,2012.0,6.0,17.0,11.0,46.0
102366,2012.0,6.0,7.0,11.0,38.0
102720,2012.0,7.0,8.0,16.0,49.0
103681,2012.0,6.0,17.0,11.0,46.0
106983,2012.0,6.0,7.0,11.0,38.0
107337,2012.0,7.0,8.0,16.0,49.0
108300,2012.0,6.0,17.0,11.0,46.0



Nombre de lignes maintenant valides après correction : 116 / 116
Exemples de lignes corrigées et maintenant valides :


,id,date_taken_year,date_taken_month,date_taken_day,date_taken_hour,date_taken_minute
42366,5464485473,2011.0,11.0,2.0,21.0,6.0
85950,6674970791,2012.0,9.0,10.0,12.0,29.0
90872,6674970791,2012.0,9.0,10.0,12.0,29.0
98816,7387000024,2012.0,6.0,17.0,11.0,46.0
102366,7583546064,2012.0,6.0,7.0,11.0,38.0
102720,7527657538,2012.0,7.0,8.0,16.0,49.0
103681,7387000024,2012.0,6.0,17.0,11.0,46.0
106983,7583546064,2012.0,6.0,7.0,11.0,38.0
107337,7527657538,2012.0,7.0,8.0,16.0,49.0
108300,7387000024,2012.0,6.0,17.0,11.0,46.0


# Nettoyage des données

In [9]:
df.columns = df.columns.str.strip()

df_clean = df.dropna(axis=1, how='all')

cols_date = ['date_taken_year', 'date_taken_month', 'date_taken_day', 'date_taken_hour', 'date_taken_minute']

df_clean = df_clean.drop_duplicates(subset=['user', 'lat', 'long'] + cols_date)
df_clean = df_clean.dropna(subset=cols_date)

for col in cols_date:
    df_clean[col] = df_clean[col].astype(int)

colonnes_utiles = ['id', 'user', 'lat', 'long', 'tags', 'title'] + cols_date
df_clean = df_clean[colonnes_utiles]

print("\nDonnées nettoyées :")
display(df_clean.head())
print("Nombre de photos restantes : " + str(len(df_clean)))

pourcentage_supprimes = ((len(df) - len(df_clean)) / len(df)) * 100
print(f"Pourcentage de données supprimées : {pourcentage_supprimes:.2f}%")


Données nettoyées :


,id,user,lat,long,tags,title,date_taken_year,date_taken_month,date_taken_day,date_taken_hour,date_taken_minute
0,4395181099,30624617@N03,45.754858,4.821710,"chair,lyon,rhône,chaise,rhônealpes",Chaises avec vue,2010,2,28,15,11
1,4394748717,35853470@N00,45.753270,4.862953,NaN,NaN,2010,2,28,17,51
2,4394694699,11817998@N05,45.760655,4.846564,"365,iphone",59/365 - R46 V103 B163,2010,2,28,17,29
3,4394803790,11545749@N06,45.784000,4.874072,"nin,nineinchnails,gift,screening,toiou,avott",2010-01-29 Toiou Avott Lyon,2010,1,28,20,15
4,4394803554,11545749@N06,45.784000,4.874072,"lyon,nin,nineinchnails,gift,screening,toiou,avott",2010-01-28 Toiou Avott Lyon,2010,1,28,20,10


Nombre de photos restantes : 131973
Pourcentage de données supprimées : 68.60%


In [14]:
# Affichage du tableau complet pour une meilleure lisibilité
display(df_clean)

,id,user,lat,long,tags,title,date_taken_year,date_taken_month,date_taken_day,date_taken_hour,date_taken_minute
0,4395181099,30624617@N03,45.754858,4.821710,"chair,lyon,rhône,chaise,rhônealpes",Chaises avec vue,2010,2,28,15,11
1,4394748717,35853470@N00,45.753270,4.862953,NaN,NaN,2010,2,28,17,51
2,4394694699,11817998@N05,45.760655,4.846564,"365,iphone",59/365 - R46 V103 B163,2010,2,28,17,29
3,4394803790,11545749@N06,45.784000,4.874072,"nin,nineinchnails,gift,screening,toiou,avott",2010-01-29 Toiou Avott Lyon,2010,1,28,20,15
4,4394803554,11545749@N06,45.784000,4.874072,"lyon,nin,nineinchnails,gift,screening,toiou,avott",2010-01-28 Toiou Avott Lyon,2010,1,28,20,10
...,...,...,...,...,...,...,...,...,...,...,...
413846,48994653443,128086472@N06,45.781863,4.875416,NaN,L’égout et les couleurs,2019,10,30,16,42
419013,45039194824,146709606@N06,45.729498,4.951550,"car,autocar,interurbain,ligne,scolaire,yutong,...",YUTONG ICe 12 - DGC,2018,10,17,15,37
419071,44824515765,14869313@N00,45.768700,4.840502,"france,auvergnerhônealpes,auvergnerhonealpes,r...","Rhône river, Lyon, France",2018,10,27,11,41
419137,44995017704,48633948@N08,45.771852,4.833115,"lyon,1714,4814we69,man,hess,nmt222,trolleybus,...",Lyon 1714 Rue Pouteau,2017,10,18,14,20


In [11]:
print(f"Année min : {df_clean['date_taken_year'].min()}")
print(f"Année max : {df_clean['date_taken_year'].max()}")

df_clean = df_clean[
    (df_clean['date_taken_year'] >= 2004) &
    (df_clean['date_taken_year'] <= 2026)
]

print(f"Année min : {df_clean['date_taken_year'].min()}")
print(f"Année max : {df_clean['date_taken_year'].max()}")
print("Après filtre temporel : " + str(len(df_clean)) + " photos.")

Année min : 1
Année max : 2238
Année min : 2004
Année max : 2019
Après filtre temporel : 131763 photos.


In [16]:
# Vérification des valeurs invalides dans les colonnes de date
print("Vérification des heures invalides (pas entre 0 et 23) :")
heures_invalides = df_clean[(df_clean['date_taken_hour'] < 0) | (df_clean['date_taken_hour'] > 23)]
print(f"Nombre d'heures invalides : {len(heures_invalides)}")
if len(heures_invalides) > 0:
    display(heures_invalides[['id', 'date_taken_hour']].head(10))  # Afficher les 10 premières pour lisibilité

print("\nVérification des mois invalides (pas entre 1 et 12) :")
mois_invalides = df_clean[(df_clean['date_taken_month'] < 1) | (df_clean['date_taken_month'] > 12)]
print(f"Nombre de mois invalides : {len(mois_invalides)}")
if len(mois_invalides) > 0:
    display(mois_invalides[['id', 'date_taken_month']].head(10))

print("\nVérification des jours invalides (pas entre 1 et 31) :")
jours_invalides = df_clean[(df_clean['date_taken_day'] < 1) | (df_clean['date_taken_day'] > 31)]
print(f"Nombre de jours invalides : {len(jours_invalides)}")
if len(jours_invalides) > 0:
    display(jours_invalides[['id', 'date_taken_day']].head(10))

Vérification des heures invalides (pas entre 0 et 23) :
Nombre d'heures invalides : 0

Vérification des mois invalides (pas entre 1 et 12) :
Nombre de mois invalides : 0

Vérification des jours invalides (pas entre 1 et 31) :
Nombre de jours invalides : 0


In [12]:
# Vérifier que les coordonnées géographiques sont cohérentes.
photos_avant_filtre = len(df_clean)

df_clean = df_clean[
    (df_clean['lat'] >= -90) & (df_clean['lat'] <= 90) &
    (df_clean['long'] >= -180) & (df_clean['long'] <= 180)
]
photos_supprimees = photos_avant_filtre - len(df_clean)
print(f"Photos supprimées par le filtre géographique : {photos_supprimees}")
print(f"Photos avec coordonnées valides : {len(df_clean)}")
print(f"Min/Max Latitude  : {df_clean['lat'].min()} / {df_clean['lat'].max()}")
print(f"Min/Max Longitude : {df_clean['long'].min()} / {df_clean['long'].max()}")

Photos supprimées par le filtre géographique : 0
Photos avec coordonnées valides : 131763
Min/Max Latitude  : 45.6552 / 45.85495
Min/Max Longitude : 4.720312 / 5.006709


In [13]:
from folium.plugins import FastMarkerCluster
# Création de la carte centrée sur une latitude et une longitude moyennes
m = folium.Map(location=[45.7640, 4.8357], zoom_start=13)
locations = list(zip(df['lat'], df['long']))
FastMarkerCluster(data=locations).add_to(m)
# Sauvegarde la carte dans un fichier HTML
m.save('ma_carte_lyon.html')
 

# Clustering